In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6652,2021-01-05T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1198,190,1388,36038,...,2564,NaN,NaN,99514,1255175,835040.0,NaN,17.0,NaN,NaN
6653,2021-01-05T17:00:00,ITA,9,Toscana,43.769231,11.255889,860,139,999,8638,...,3762,NaN,NaN,122420,1917959,1073738.0,NaN,8.0,Positivi diagnosticati solo con test antigenic...,NaN
6654,2021-01-05T17:00:00,ITA,10,Umbria,43.106758,12.388247,275,43,318,3615,...,640,NaN,NaN,29728,513892,255640.0,NaN,4.0,NaN,NaN
6655,2021-01-05T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,54,1,55,353,...,387,NaN,NaN,7380,63793,39235.0,NaN,0.0,NaN,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6652,2021-01-05T17:00:00,19,Sicilia,1198,190,1388,36038,37426,848,1576,59524,2564,NaN,NaN,99514,1255175,835040.0,17.0,NaN,NaN
6653,2021-01-05T17:00:00,9,Toscana,860,139,999,8638,9637,-342,337,109021,3762,NaN,NaN,122420,1917959,1073738.0,8.0,Positivi diagnosticati solo con test antigenic...,NaN
6654,2021-01-05T17:00:00,10,Umbria,275,43,318,3615,3933,55,234,25155,640,NaN,NaN,29728,513892,255640.0,4.0,NaN,NaN
6655,2021-01-05T17:00:00,2,Valle d'Aosta,54,1,55,353,408,-2,21,6585,387,NaN,NaN,7380,63793,39235.0,0.0,NaN,NaN


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-01-05', '2021-01-04')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2021-01-05T17:00:00,1,Piemonte,2847,187,3034,16608,19642,-871,1109,173930,8043,NaN,NaN,201615,1704739,994333.0,15.0,NaN,NaN
1,2021-01-05T17:00:00,2,Valle d'Aosta,54,1,55,353,408,-2,21,6585,387,NaN,NaN,7380,63793,39235.0,0.0,NaN,NaN
2,2021-01-05T17:00:00,3,Lombardia,3344,475,3819,48498,52317,-370,1338,409548,25406,NaN,NaN,487271,4930952,2634038.0,37.0,NaN,NaN
3,2021-01-05T17:00:00,5,Veneto,2694,361,3055,91173,94228,-728,3151,168881,6988,NaN,NaN,270097,3374382,1235218.0,32.0,NaN,NaN
4,2021-01-05T17:00:00,6,Friuli Venezia Giulia,657,59,716,11078,11794,101,688,38704,1772,NaN,NaN,52270,954564,346775.0,3.0,NaN,NaN
5,2021-01-05T17:00:00,7,Liguria,708,73,781,4791,5572,-176,404,53412,2929,NaN,NaN,61913,724539,327367.0,2.0,I 168529 test antigenici rapidi sono riferiti ...,NaN
6,2021-01-05T17:00:00,8,Emilia-Romagna,2696,232,2928,54249,57177,792,1506,115907,8005,NaN,NaN,181089,2628108,1246273.0,18.0,NaN,NaN
7,2021-01-05T17:00:00,9,Toscana,860,139,999,8638,9637,-342,337,109021,3762,NaN,NaN,122420,1917959,1073738.0,8.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2021-01-05T17:00:00,10,Umbria,275,43,318,3615,3933,55,234,25155,640,NaN,NaN,29728,513892,255640.0,4.0,NaN,NaN
9,2021-01-05T17:00:00,11,Marche,495,65,560,12146,12706,389,458,29447,1634,NaN,NaN,43787,550075,326877.0,3.0,NaN,NaN


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2021-01-04T17:00:00,1,Piemonte,2819,183,3002,17511,20513,-950,446,172005,7988,NaN,NaN,200506,1697788,991545.0,3.0,NaN,NaN
1,2021-01-04T17:00:00,2,Valle d'Aosta,63,1,64,346,410,-4,27,6562,387,NaN,NaN,7359,63531,38800.0,0.0,NaN,NaN
2,2021-01-04T17:00:00,3,Lombardia,3227,484,3711,48976,52687,-4268,863,407902,25344,NaN,NaN,485933,4918162,2629427.0,22.0,NaN,NaN
3,2021-01-04T17:00:00,5,Veneto,2706,368,3074,91882,94956,-203,1682,165177,6813,NaN,NaN,266946,3356648,1230879.0,20.0,NaN,NaN
4,2021-01-04T17:00:00,6,Friuli Venezia Giulia,663,63,726,10967,11693,44,281,38158,1731,NaN,NaN,51582,947459,345095.0,7.0,NaN,NaN
5,2021-01-04T17:00:00,7,Liguria,716,75,791,4957,5748,-39,204,52844,2917,NaN,NaN,61509,719953,326003.0,5.0,I 165501 test antigenici rapidi sono riferiti ...,NaN
6,2021-01-04T17:00:00,8,Emilia-Romagna,2684,235,2919,53466,56385,-243,1600,115260,7941,NaN,NaN,179586,2612313,1242760.0,12.0,NaN,NaN
7,2021-01-04T17:00:00,9,Toscana,877,141,1018,8961,9979,31,313,108360,3744,NaN,NaN,122083,1910036,1071154.0,8.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2021-01-04T17:00:00,10,Umbria,274,44,318,3560,3878,-26,54,24980,636,NaN,NaN,29494,509643,254520.0,2.0,NaN,NaN
9,2021-01-04T17:00:00,11,Marche,487,65,552,11765,12317,251,355,29397,1615,NaN,NaN,43329,545237,323629.0,3.0,NaN,NaN


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2021-01-05T17:00:00,1,Piemonte,2847,187,3034,16608,19642,-871,1109,173930,8043,NaN,NaN,201615,1704739,994333.0,15.0,NaN,NaN
1,2021-01-05T17:00:00,2,Valle d'Aosta,54,1,55,353,408,-2,21,6585,387,NaN,NaN,7380,63793,39235.0,0.0,NaN,NaN
2,2021-01-05T17:00:00,3,Lombardia,3344,475,3819,48498,52317,-370,1338,409548,25406,NaN,NaN,487271,4930952,2634038.0,37.0,NaN,NaN
3,2021-01-05T17:00:00,4,Trentino-Alto Adige,574,72,646,12343,12989,100,376,38100,1742,0.0,0.0,52831,826239,317354.0,7.0,0,NaN
4,2021-01-05T17:00:00,5,Veneto,2694,361,3055,91173,94228,-728,3151,168881,6988,NaN,NaN,270097,3374382,1235218.0,32.0,NaN,NaN
5,2021-01-05T17:00:00,6,Friuli Venezia Giulia,657,59,716,11078,11794,101,688,38704,1772,NaN,NaN,52270,954564,346775.0,3.0,NaN,NaN
6,2021-01-05T17:00:00,7,Liguria,708,73,781,4791,5572,-176,404,53412,2929,NaN,NaN,61913,724539,327367.0,2.0,I 168529 test antigenici rapidi sono riferiti ...,NaN
7,2021-01-05T17:00:00,8,Emilia-Romagna,2696,232,2928,54249,57177,792,1506,115907,8005,NaN,NaN,181089,2628108,1246273.0,18.0,NaN,NaN
8,2021-01-05T17:00:00,9,Toscana,860,139,999,8638,9637,-342,337,109021,3762,NaN,NaN,122420,1917959,1073738.0,8.0,Positivi diagnosticati solo con test antigenic...,NaN
9,2021-01-05T17:00:00,10,Umbria,275,43,318,3615,3933,55,234,25155,640,NaN,NaN,29728,513892,255640.0,4.0,NaN,NaN


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,...,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,0,2021-01-04T17:00:00,1,Piemonte,2819,183,3002,17511,20513,-950,...,172005,7988,NaN,NaN,200506,1697788,991545.0,3.0,NaN,NaN
1,1,2021-01-04T17:00:00,2,Valle d'Aosta,63,1,64,346,410,-4,...,6562,387,NaN,NaN,7359,63531,38800.0,0.0,NaN,NaN
2,2,2021-01-04T17:00:00,3,Lombardia,3227,484,3711,48976,52687,-4268,...,407902,25344,NaN,NaN,485933,4918162,2629427.0,22.0,NaN,NaN
3,19,2021-01-04T17:00:00,4,Trentino-Alto Adige,576,68,644,12245,12889,58,...,37834,1732,0.0,0.0,52455,823039,316690.0,2.0,0,NaN
4,3,2021-01-04T17:00:00,5,Veneto,2706,368,3074,91882,94956,-203,...,165177,6813,NaN,NaN,266946,3356648,1230879.0,20.0,NaN,NaN
5,4,2021-01-04T17:00:00,6,Friuli Venezia Giulia,663,63,726,10967,11693,44,...,38158,1731,NaN,NaN,51582,947459,345095.0,7.0,NaN,NaN
6,5,2021-01-04T17:00:00,7,Liguria,716,75,791,4957,5748,-39,...,52844,2917,NaN,NaN,61509,719953,326003.0,5.0,I 165501 test antigenici rapidi sono riferiti ...,NaN
7,6,2021-01-04T17:00:00,8,Emilia-Romagna,2684,235,2919,53466,56385,-243,...,115260,7941,NaN,NaN,179586,2612313,1242760.0,12.0,NaN,NaN
8,7,2021-01-04T17:00:00,9,Toscana,877,141,1018,8961,9979,31,...,108360,3744,NaN,NaN,122083,1910036,1071154.0,8.0,Positivi diagnosticati solo con test antigenic...,NaN
9,8,2021-01-04T17:00:00,10,Umbria,274,44,318,3560,3878,-26,...,24980,636,NaN,NaN,29494,509643,254520.0,2.0,NaN,NaN


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione
0,2021-01-05T17:00:00,1,Piemonte,2847,187,3034,16608,19642,-871,1109,...,8043,NaN,NaN,201615,1704739,994333.0,15.0,NaN,NaN,4341375
1,2021-01-05T17:00:00,2,Valle d'Aosta,54,1,55,353,408,-2,21,...,387,NaN,NaN,7380,63793,39235.0,0.0,NaN,NaN,125501
2,2021-01-05T17:00:00,3,Lombardia,3344,475,3819,48498,52317,-370,1338,...,25406,NaN,NaN,487271,4930952,2634038.0,37.0,NaN,NaN,10103969
3,2021-01-05T17:00:00,4,Trentino-Alto Adige,574,72,646,12343,12989,100,376,...,1742,0.0,0.0,52831,826239,317354.0,7.0,0,NaN,1074819
4,2021-01-05T17:00:00,5,Veneto,2694,361,3055,91173,94228,-728,3151,...,6988,NaN,NaN,270097,3374382,1235218.0,32.0,NaN,NaN,4907704
5,2021-01-05T17:00:00,6,Friuli Venezia Giulia,657,59,716,11078,11794,101,688,...,1772,NaN,NaN,52270,954564,346775.0,3.0,NaN,NaN,1211357
6,2021-01-05T17:00:00,7,Liguria,708,73,781,4791,5572,-176,404,...,2929,NaN,NaN,61913,724539,327367.0,2.0,I 168529 test antigenici rapidi sono riferiti ...,NaN,1543127
7,2021-01-05T17:00:00,8,Emilia-Romagna,2696,232,2928,54249,57177,792,1506,...,8005,NaN,NaN,181089,2628108,1246273.0,18.0,NaN,NaN,4467118
8,2021-01-05T17:00:00,9,Toscana,860,139,999,8638,9637,-342,337,...,3762,NaN,NaN,122420,1917959,1073738.0,8.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729
9,2021-01-05T17:00:00,10,Umbria,275,43,318,3615,3933,55,234,...,640,NaN,NaN,29728,513892,255640.0,4.0,NaN,NaN,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione,totale_posti_terapia_intensiva
0,2021-01-05T17:00:00,1,Piemonte,2847,187,3034,16608,19642,-871,1109,...,NaN,NaN,201615,1704739,994333.0,15.0,NaN,NaN,4341375,575
1,2021-01-05T17:00:00,2,Valle d'Aosta,54,1,55,353,408,-2,21,...,NaN,NaN,7380,63793,39235.0,0.0,NaN,NaN,125501,20
2,2021-01-05T17:00:00,3,Lombardia,3344,475,3819,48498,52317,-370,1338,...,NaN,NaN,487271,4930952,2634038.0,37.0,NaN,NaN,10103969,1036
3,2021-01-05T17:00:00,4,Trentino-Alto Adige,574,72,646,12343,12989,100,376,...,0.0,0.0,52831,826239,317354.0,7.0,0,NaN,1074819,106
4,2021-01-05T17:00:00,5,Veneto,2694,361,3055,91173,94228,-728,3151,...,NaN,NaN,270097,3374382,1235218.0,32.0,NaN,NaN,4907704,1016
5,2021-01-05T17:00:00,6,Friuli Venezia Giulia,657,59,716,11078,11794,101,688,...,NaN,NaN,52270,954564,346775.0,3.0,NaN,NaN,1211357,180
6,2021-01-05T17:00:00,7,Liguria,708,73,781,4791,5572,-176,404,...,NaN,NaN,61913,724539,327367.0,2.0,I 168529 test antigenici rapidi sono riferiti ...,NaN,1543127,209
7,2021-01-05T17:00:00,8,Emilia-Romagna,2696,232,2928,54249,57177,792,1506,...,NaN,NaN,181089,2628108,1246273.0,18.0,NaN,NaN,4467118,634
8,2021-01-05T17:00:00,9,Toscana,860,139,999,8638,9637,-342,337,...,NaN,NaN,122420,1917959,1073738.0,8.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729,523
9,2021-01-05T17:00:00,10,Umbria,275,43,318,3615,3933,55,234,...,NaN,NaN,29728,513892,255640.0,4.0,NaN,NaN,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-01-05T17:00:00,5,Veneto,2694,361,3055,91173,94228,-728,3151,...,35.53,-709,1469,17734,4339.0,175,3704,17.77,0.06421,5.50
1,2021-01-05T17:00:00,12,Lazio,2869,308,3177,74131,77308,231,1719,...,36.36,224,385,13132,6789.0,72,1416,13.09,0.02931,2.91
2,2021-01-05T17:00:00,19,Sicilia,1198,190,1388,36038,37426,848,1576,...,32.31,827,185,9537,6290.0,36,692,16.53,0.03172,2.00
3,2021-01-05T17:00:00,8,Emilia-Romagna,2696,232,2928,54249,57177,792,1506,...,36.59,783,-94,15795,3513.0,64,647,9.53,0.03371,4.05
4,2021-01-05T17:00:00,3,Lombardia,3344,475,3819,48498,52317,-370,1338,...,45.85,-478,475,12790,4611.0,62,1646,10.46,0.01324,4.82
5,2021-01-05T17:00:00,1,Piemonte,2847,187,3034,16608,19642,-871,1109,...,32.52,-903,663,6951,2788.0,55,1925,15.95,0.02554,4.64
6,2021-01-05T17:00:00,16,Puglia,1401,152,1553,52027,53580,41,1081,...,41.19,89,450,10273,4540.0,14,1026,10.52,0.02697,2.38
7,2021-01-05T17:00:00,15,Campania,1372,97,1469,74891,76360,-1137,688,...,19.21,-1133,-59,7425,4193.0,33,1792,9.27,0.01189,3.35
8,2021-01-05T17:00:00,6,Friuli Venezia Giulia,657,59,716,11078,11794,101,688,...,32.78,111,407,7105,1680.0,41,546,9.68,0.05680,4.31
9,2021-01-05T17:00:00,11,Marche,495,65,560,12146,12706,389,458,...,45.45,381,103,4838,3248.0,19,50,9.47,0.03016,2.88


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Trentino-Alto Adige,72,34,106,67.92
1,Lombardia,475,561,1036,45.85
2,Marche,65,78,143,45.45
3,Puglia,152,217,369,41.19
4,Umbria,43,74,117,36.75
5,Emilia-Romagna,232,402,634,36.59
6,Lazio,308,539,847,36.36
7,Veneto,361,655,1016,35.53
8,Liguria,73,136,209,34.93
9,Friuli Venezia Giulia,59,121,180,32.78


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629,2021-01-03T17:00:00,22,P.A. Trento,382,44,426,1555,1981,-1,130,19337,961,NaN,NaN,22279,456111,152347.0,4.0,NaN,"Dei 130 casi odierni, 37 sono molecolari con p..."
630,2021-01-04T17:00:00,21,P.A. Bolzano,197,25,222,10602,10824,-26,37,18403,756,NaN,NaN,29983,366281,164012.0,1.0,NaN,NaN
631,2021-01-04T17:00:00,22,P.A. Trento,379,43,422,1643,2065,84,193,19431,976,NaN,NaN,22472,456758,152678.0,1.0,NaN,"Dei 193 casi odierni, 151 sono molecolari con ..."
632,2021-01-05T17:00:00,21,P.A. Bolzano,203,27,230,10683,10913,89,148,18460,758,NaN,NaN,30131,367689,164243.0,2.0,NaN,NaN


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,2021-01-01T17:00:00,4,Trentino-Alto Adige,521,66,587,12105,12692,161,463,...,13.0,4978.0,960.0,181.0,11.24,4.62,9.30,48.23,-2.94,1.28
313,2021-01-02T17:00:00,4,Trentino-Alto Adige,545,65,610,12210,12820,128,202,...,10.0,1302.0,359.0,105.0,10.66,4.76,15.51,56.27,-1.52,1.01
314,2021-01-03T17:00:00,4,Trentino-Alto Adige,568,68,636,12195,12831,11,226,...,8.0,2855.0,570.0,-15.0,10.69,4.96,7.92,39.65,4.62,0.09
315,2021-01-04T17:00:00,4,Trentino-Alto Adige,576,68,644,12245,12889,58,230,...,20.0,1852.0,647.0,50.0,10.56,5.00,12.42,35.55,0.00,0.45


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
